<a href="https://colab.research.google.com/github/ADRIANATORREZANI/ADRIANATORREZANI/blob/main/Adriana_Chat06.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install langchain langchain-openai langchain-community openai faiss-cpu pypdf gradio python-dotenv pandas openpyxl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 34.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 57.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.3/302.3 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 MB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.6/322.6 kB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 109.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 55.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB

In [2]:
!pip install python-dotenv --quiet

In [4]:
!pip install langchain-community --quiet

In [3]:
!pip install -U langchain langchain-community --quiet

In [5]:
!pip install gradio --quiet

In [6]:
!pip install -U langchain-openai --quiet

In [7]:
pip install pandas openpyxl --quiet

In [8]:
pip install sentence-transformers --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 63.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 35.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 61.3 MB/s eta 0:00:00


In [12]:
import os
import sqlite3
import pandas as pd
from dotenv import load_dotenv
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_openai import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
import gradio as gr

# Carrega chave da API
load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENROUTER_API_KEY")
os.environ["OPENAI_API_BASE"] = "https://openrouter.ai/api/v1"
# Carrega e divide o PDF
loader = PyPDFLoader("apostila_python.pdf")
docs = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
documents = text_splitter.split_documents(docs)

# Cria embeddings com modelo local (sem necessidade de API)
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
vectordb = FAISS.from_documents(documents, embeddings)
retriever = vectordb.as_retriever()

# Cria banco de dados SQLite se não existir
conn = sqlite3.connect("historico_conversas.db")
cursor = conn.cursor()
cursor.execute('''
CREATE TABLE IF NOT EXISTS conversas (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    aluno TEXT,
    pergunta TEXT,
    resposta TEXT,
    timestamp DATETIME DEFAULT CURRENT_TIMESTAMP
)
''')
conn.commit()
conn.close()

# LLM da OpenRouter
llm = ChatOpenAI(model="deepseek/deepseek-r1:free", temperature=0.4)

# Memória da conversa
memoria = ConversationBufferMemory(memory_key="chat_history", return_messages=True, output_key="answer")

# Cadeia com recuperação de contexto e memória
qa_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retriever,
    memory=memoria,
    return_source_documents=True,
    output_key="answer"
)

# Função para salvar no banco
def salvar_conversa(nome, pergunta, resposta):
    conn = sqlite3.connect("historico_conversas.db")
    cursor = conn.cursor()
    cursor.execute("INSERT INTO conversas (aluno, pergunta, resposta) VALUES (?, ?, ?)",
                   (nome or "Anônimo", pergunta, resposta))
    conn.commit()
    conn.close()

# Função para responder usando o PDF e memória
def responder(pergunta, nome):
    try:
        resultado = qa_chain.invoke({"question": pergunta})
        resposta = resultado["answer"]
        fontes = resultado.get("source_documents", [])

        # Fallback se nenhuma fonte relevante for encontrada
        if not fontes or all(len(doc.page_content.strip()) == 0 for doc in fontes):
            resposta = llm.invoke(pergunta)

        salvar_conversa(nome, pergunta, resposta)
        return resposta

    except Exception as e:
        import traceback
        return f"❌ Erro:\n{traceback.format_exc()}"

# Resetar memória
def resetar_memoria():
    memoria.clear()
    return "✅ Memória resetada com sucesso!"

# Exportar conversas para arquivos
def exportar_conversas():
    conn = sqlite3.connect("historico_conversas.db")
    df = pd.read_sql_query("SELECT * FROM conversas ORDER BY timestamp DESC", conn)
    df.to_csv("conversas_exportadas.csv", index=False)
    df.to_excel("conversas_exportadas.xlsx", index=False, engine="openpyxl")
    conn.close()
    return "✅ Arquivos 'conversas_exportadas.csv' e 'conversas_exportadas.xlsx' foram salvos!"

# Interface Gradio
with gr.Blocks() as app:
    gr.Markdown("## 🤖 Tutor de Python com IA + Apostila PDF + Histórico em BD")
    nome = gr.Textbox(label="Seu nome (opcional)", placeholder="Ex: João")
    pergunta = gr.Textbox(label="Sua dúvida sobre Python", placeholder="Ex: Como usar listas em Python?")
    resposta = gr.Textbox(label="Resposta do Assistente")

    botao_enviar = gr.Button("Enviar")
    botao_resetar = gr.Button("🔁 Resetar Memória")
    botao_exportar = gr.Button("📤 Exportar Histórico")

    botao_enviar.click(fn=responder, inputs=[pergunta, nome], outputs=resposta)
    botao_resetar.click(fn=resetar_memoria, outputs=resposta)
    botao_exportar.click(fn=exportar_conversas, outputs=resposta)

app.launch(share=True)


<ipython-input-12-7c5c1eceab59>:48: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memoria = ConversationBufferMemory(memory_key="chat_history", return_messages=True, output_key="answer")


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://82d51621fec1a9c66b.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
